\section{Kitaev Hamiltonian}

\begin{equation}
    H_K = - \sum_{<j,k>_\alpha} J_\alpha S_j^\alpha S_k^\alpha
\end{equation}
where $\alpha = x, y, z$.

\subsection{Energy spectrum of Kitaev Hamiltonian}

The ground state (GS) of Kitaev model can be exactly solved by representing the spin operators by Majorana operators: $\sigma_j^\alpha = ic_j b_j^\alpha$. The Hamiltonain becomes:

\begin{equation}
    H_{Maj} = \sum_{<j,k>_\alpha} \frac{J_\alpha}{4} (i b_j^\alpha b_k^\alpha ) \ i  c_j c_k 
            = \sum_{<j,k>_\alpha} \frac{J_\alpha}{4} u_{<j,k>_\alpha} \ i c_j c_k
\end{equation}

where $u_{<j,k>_\alpha} = i b_j^\alpha b_k^\alpha$. There also be a constrain $i c_j b_j^x b_j^y b_j^z = 1$ applied to limit the system to the physical Hilbert space. It has been proved that all $u_{<j,k>_\alpha} = 1$ in the ground state. Thus the Hamiltonian becomes quadratic and the energy spectrum can be solved by Fourier transformation. The GS energy spectrum is:

\begin{align}
    E(\vec{k}) &= \pm |f(\vec{k})| \\
    f(\vec{k}) &= \frac{1}{2} (J_x e^{i k_1} + J_y e^{i k_2} + J_z )
\end{align}

where $k_1$ and $k_2$ are the components of $\vec{k}$ along $\vec{n}_1$ and $\vec{n}_2$ directions.


\subsection{Ground state energy of a finite lattice}

In order to check the correctness of the ED code. Here we give a explicite calculation of the GS energy of Kitaev model on $N_1 \times N_2 \times 2$ lattice. The lattice vectors are $\vec{n}_1 = (0,1), \vec{n}_2 = (\frac{1}{2},\frac{\sqrt{3}}{2})$. The number of unit cells along these two directions are $N_1$ and $N_2$ respectively.

The GS Hamiltonian is:
\begin{equation}
    H_G = \sum_{<j,k>_\alpha} \frac{J_\alpha}{4} i c_j c_k
\end{equation}

We can explicitly write $H_G$ in real space and diagonalize the Hamiltonian to get its ground state energy.

In the following code, the basis we use is $(c_{1A},\ c_{1B},\ ... \,\ c_{NA},\ c_{NB})^T$, where $N = N_1 \times N_2$ is the number of lattice sites. We label the A sublattice atom in the $n$-th unit cell by $2n$ the other by $2n + 1$.

In [33]:
import numpy as np

def KitaevReal(Jx, Jy, Jz, periodic = True):
    '''Real space Kitaev Hamiltonian on N1 * N2 * 2 lattice
    
       Setups required: 
           N1, N2: width and length of the rhomboid cluster
           SiteNum: total site number
        
        Return: Hr : Hamiltonian matrix
    '''
    Hr = np.zeros([SiteNum, SiteNum])
    Jx = Jx/4
    Jy = Jy/4
    Jz = Jz/4
    
    for i in range(N1*N2):
        '''Sx Sx'''
        Hr[2*i, 2*i+1] += Jx /2
        Hr[2*i + 1, 2*i] +=  - Jx /2
            
        '''Sy Sy'''
        if i % N1 != 0:
            next = 2*i -1
        elif i % N1 == 0 and periodic == True:
            next = 2 * (i + N1) -1
        else:
            next = 0.5
        if next % 1 == 0:
            Hr[2*i, next] +=  Jy /2
            Hr[next, 2*i] +=  -Jy /2
  
        '''Sz Sz'''
        if i >= N1:
            next = 2*(i - N1) +1
        elif i < N1 and periodic == True:
            next = 2 * (i + N1 * N2 - N1) +1
        else:
            next = 0.5
        if next % 1 == 0:
            Hr[2*i, next] +=  Jz /2
            Hr[next, 2*i] +=  -Jz /2
    Hr = 1j * Hr
    return Hr

N1 = 1
N2 = 2

Jx = 1
Jy = 1
Jz = 1
SiteNum = N1 * N2 * 2

Hr = np.array([SiteNum, SiteNum])
Hr = KitaevReal(Jx, Jy, Jz, periodic = True)
evals, evecs = np.linalg.eigh(Hr)
sum = 0
for i in range(N1*N2):
    sum+= evals[i] 
print(evals)
print(sum * 2)

[-0.375 -0.125  0.125  0.375]
-0.9999999999999998


\section{Basis generation}

We use the eigen states of course we $S^z$ as basis states and use $|0 \rangle$ to represent $|\downarrow \rangle$ and $|1 \rangle$ to represent $|\uparrow \rangle$. The states can thus be represented by a set of binary numbers. Moreover, we can use the corresponding dicimal number of each state as its tag.

For a system with N sites, the dimension of Hilbert space is $2^N$.

\section{Hamiltonian matrix}

The Hamiltonian can be rewriten to $H_K = H_x + H_y +H_z$, where $H_z$ is the diagonal term and $H_x, H_y$ are off-diagonal terms:
\begin{align}
      H_x &= -\sum_{<i,j>_x} J_x S_i^x S_j^x = -\sum_{<i,j>_x} \frac{J_x}{4} ( S_i^+ S_j^- + S_i^- S_j^+ + S_i^+ S_j^+            + S_i^- S_j^-)\\
      H_y &= -\sum_{<i,j>_y} J_y S_i^y S_j^y = -\sum_{<i,j>_y} \frac{J_y}{4} ( S_i^+ S_j^- + S_i^- S_j^+ - S_i^+ S_j^+            - S_i^- S_j^-)\\
      H_z &= -\sum_{<i,j>_z} J_z S_i^z S_j^z
\end{align}
where $S_i^\pm = S_i^x \pm i S_i^y$.

\subsection{$ N_1 \times N_2 \times 2$ lattice}

The lattice vectors are $\vec{n}_1 = (0,1), \vec{n}_2 = (\frac{1}{2},\frac{\sqrt{3}}{2})$. The number of unit cells along these two directions are $N_1$ and $N_2$ respectively. We label each unit cell by its row and colume numbers $(r, c)$(start from $0$), the label of this unit cell given by $ n = r \times N_1 + c$. We can label A atom in the $n$-th unit cell as $2n$ and B atom as $2n + 1$.

In [3]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigsh

def StateConfig(tag):
    '''The configuration of the state with tag.
    
       Setups required: 
           dim: Hilbert space dimension
           SiteNum: total site number
           
       Return: binary form of tag. type: list'''
    if tag >= dim:
        return print('Error: The tag is out of range.')
    else:
        b = bin(tag)[2:]
        b = b.rjust(SiteNum,'0') 
        return b

def flip(tag,i,j):
    '''Flip the spin on i,j site.
    
       Setups required: 
           SiteNum: total site number
           
       Return: The tag of new state, type: int '''
    f = pow(2, SiteNum - i -1) + pow(2, SiteNum - j -1)
    Newtag = tag^f
    return Newtag

def KitaevRhom(Jx, Jy, Jz, periodic = False):
    '''Kitaev Hamiltonian on N1 * N2 * 2 lattice
    
       Setups required: 
           N1, N2: width and length of the rhomboid cluster
           SiteNum: total site number
           dim: Hilbert space dimension
        
        Return: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        '''
    numNZero = 0 # count the number of non-zero element
    H = []
    row = []
    col = []
    for tag in range(dim):
        #print('The state numnber is: ', tag)
        si = StateConfig(tag) # initial state configuration
        #print('initial configuration=',si,', tag = ',tag)
        
        temp = {} # A temporay dict
        
        for r in range(N2):
            for c in range(N1):
                n = r * N1 + c #label of unit cell
                '''Sx Sx'''
                next = 2*n + 1
                
                ntag = flip(tag, 2*n, next)
                if ntag in temp:
                    temp[ntag] += -Jx /4
                else:
                    temp[ntag] = -Jx /4
                
                '''Sz-Sz on X-direction'''
                #if tag in temp:
                    #temp[tag] += - Jz * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                #else:
                    #temp[tag] = -Jz * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                
                '''Sy-Sy on X-direction'''
                #ntag = flip(tag, 2*n, next)
                #if ntag in temp:
                    #temp[ntag] += Jy * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                #else:
                    #temp[ntag] = Jy * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                
                '''Sy-Sy on X-direction: new'''
                #ntag = flip(tag, 2*n, next)
                #if ntag in temp:
                    #if si[2*n] == si[next]:
                        #temp[ntag] += Jy /4
                    #else:
                        #temp[ntag] -= Jy /4
                #else:
                    #if si[2*n] == si[next]:
                        #temp[ntag] = Jy /4
                    #else:
                        #temp[ntag] = -Jy /4
                
                
                '''Sy Sy'''
                if c > 0:
                    next = 2*n -1
                elif c == 0 and periodic == True:
                    next = 2 * (n + N1) -1
                else:
                    next = 0.5
                    
                if next % 1 == 0:
                    ntag = flip(tag, 2*n, next)
                    if ntag in temp:
                        temp[ntag] += Jy * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                    else:
                        temp[ntag] = Jy * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                    
                    '''Sy-Sy new'''
                    #ntag = flip(tag, 2*n, next)
                    #if ntag in temp:
                        #if si[2*n] == si[next]:
                            #temp[ntag] += Jy /4
                        #else:
                            #temp[ntag] -= Jy /4
                    #else:
                        #if si[2*n] == si[next]:
                            #temp[ntag] = Jy /4
                        #else:
                            #temp[ntag] = -Jy /4
                            
                    '''Sz-Sz on Y-direction'''
                    #if tag in temp:
                        #temp[tag] += -Jz * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                    #else:
                        #temp[tag] = -Jz * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                    
                    '''Sx-Sx on Y-direction'''
                    #ntag = flip(tag, 2*n, next)
                    #if ntag in temp:
                        #temp[ntag] += -Jx /4
                    #else:
                        #temp[ntag] = -Jx /4
                        

                '''Sz Sz'''
                if r > 0:
                    next = 2*(n - N1) + 1
                elif r == 0 and periodic == True:
                    next = 2 * (n + N1 * N2 - N1) +1
                else:
                    next = 0.5
                
                if next % 1 == 0:
                    if tag in temp:
                        temp[tag] += - Jz * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                    else:
                        temp[tag] = - Jz * (int(si[2*n]) - 0.5)* (int(si[next]) - 0.5)
                    

                    '''Sx-Sx on Z-direction'''
                    #ntag = flip(tag, 2*n, next)
                    #if ntag in temp:
                        #temp[ntag] += - Jx /4
                    #else:
                        #temp[ntag] = - Jx /4
                    
                    '''Sy-Sy on Z-direction'''
                    #ntag = flip(tag, 2*n, next)
                    #if ntag in temp:
                        #temp[ntag] += Jy * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                    #else:
                        #temp[ntag] = Jy * (int(si[2*n]) - 0.5) * (int(si[next]) - 0.5)
                    
                    '''Sy-Sy on Z-direction: new'''
                    #ntag = flip(tag, 2*n, next)
                    #if ntag in temp:
                        #if si[2*n] == si[next]:
                            #temp[ntag] += Jy /4
                        #else:
                            #temp[ntag] -= Jy /4
                    #else:
                        #if si[2*n] == si[next]:
                            #temp[ntag] = Jy /4
                        #else:
                            #temp[ntag] = -Jy /4
                        
        odTemp = sorted(temp)
        for sf in odTemp:
            row.append (tag)
            col.append (sf)
            H.append (temp[sf])
            numNZero += 1
            
    print ('non zero elemens number = ', numNZero)
    print('Percentage = ', numNZero/(dim * dim) * 100,'%')
    return H, row, col


N1 = 2
N2 = 3
SiteNum = N1 * N2 * 2
dim = pow(2, SiteNum)

Jx = 1
Jy = 1
Jz = 1

H = []
row = []
col = []

H, row, col = KitaevRhom(Jx, Jy, Jz, periodic = True)
#for l in range(len(H)):
    #print('(',row[l],',',col[l],') = ', H[l])

ham = csc_matrix ((H, (row, col)), shape=(dim, dim))
vals, vecs = eigsh (ham, k=5, which = 'SA')

print (vals / SiteNum)

non zero elemens number =  53248
Percentage =  0.3173828125 %
[-0.20417256 -0.20417256 -0.20417256 -0.20417256 -0.19523157]


The other way to generate the Hamiltonain matrix is to write it as the direct product of 